In [1]:
%pip install ollama
%pip install openai

# Load OPENAI_API_KEY from .env file
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from openai import OpenAI
import ollama

In [3]:
client = OpenAI()

In [4]:
def openai_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [5]:
def local_embedding(text, model='mxbai-embed-large'):
    result = ollama.embeddings(model=model, prompt=text)
    return result['embedding']

In [6]:
blurb = "The quick brown fox boldly jumps over the sleeping wolf."

In [11]:
%time v = openai_embedding(blurb)

CPU times: user 13.7 ms, sys: 2.56 ms, total: 16.2 ms
Wall time: 242 ms


In [12]:
%time v = local_embedding(blurb)

CPU times: user 1.82 ms, sys: 1.19 ms, total: 3.01 ms
Wall time: 63.3 ms


In [13]:
def process_text_segments(file_path, segment_size, fn):
    with open(file_path, 'r') as file:
        text = file.read()
    words = text.split()
    segments = [' '.join(words[i:i + segment_size]) for i in range(0, len(words), segment_size)]
    results = [fn(segment) for segment in segments]
    return results

In [16]:
pericles = "..//data/pericles.txt"

%time vs = process_text_segments(pericles, 100, openai_embedding)

CPU times: user 418 ms, sys: 16.8 ms, total: 434 ms
Wall time: 7 s


In [17]:
%time vs = process_text_segments(pericles, 100, local_embedding)

CPU times: user 68.9 ms, sys: 5.52 ms, total: 74.4 ms
Wall time: 2.42 s


In [19]:
%time vs = process_text_segments(pericles, 100, lambda s: local_embedding(s, model='nomic-embed-text'))

CPU times: user 62.2 ms, sys: 5.07 ms, total: 67.3 ms
Wall time: 1.21 s


In [20]:
segments = process_text_segments(pericles, 100, lambda s: s)
len(segments)

29

In [21]:
%time vs = client.embeddings.create(input = segments, model='text-embedding-3-small').data[0].embedding

CPU times: user 144 ms, sys: 5.99 ms, total: 150 ms
Wall time: 1.57 s
